<a href="https://colab.research.google.com/github/jitendragithub160892/Chroma_DB_with_Langchain/blob/main/Chroma_DB_with_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install  openai langchain sentence_transformers -q
!pip install chromadb -q

In [2]:
!pip install unstructured -q

In [3]:
import langchain

In [4]:

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
from langchain.document_loaders import DirectoryLoader
directory = '/content/drive/MyDrive/pets'



In [6]:
def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents



In [7]:
documents = load_docs(directory)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
len(documents)

5

In [9]:
documents

[Document(page_content='Training is essential for a harmonious life with pets, particularly for dogs. It helps pets understand their boundaries and makes cohabitation easier for both pets and owners. Training should be based on positive reinforcement. Understanding pet behavior is also important, as changes in behavior can often be a sign of underlying health issues.', metadata={'source': '/content/drive/MyDrive/pets/Training and Behaviour of Pets.txt'}),
 Document(page_content='Pets offer more than just companionship; they provide emotional support, reduce stress, and can even help their owners lead healthier lives. The bond between pets and their owners is strong, and many people consider their pets as part of the family. This bond can be especially important in times of personal or societal stress, providing comfort and consistency.', metadata={'source': '/content/drive/MyDrive/pets/The Emotional Bond Between Humans and Pets.txt'}),
 Document(page_content='Routine health care is cru

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents,chunk_size=1000,chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))

5


In [11]:
docs

[Document(page_content='Training is essential for a harmonious life with pets, particularly for dogs. It helps pets understand their boundaries and makes cohabitation easier for both pets and owners. Training should be based on positive reinforcement. Understanding pet behavior is also important, as changes in behavior can often be a sign of underlying health issues.', metadata={'source': '/content/drive/MyDrive/pets/Training and Behaviour of Pets.txt'}),
 Document(page_content='Pets offer more than just companionship; they provide emotional support, reduce stress, and can even help their owners lead healthier lives. The bond between pets and their owners is strong, and many people consider their pets as part of the family. This bond can be especially important in times of personal or societal stress, providing comfort and consistency.', metadata={'source': '/content/drive/MyDrive/pets/The Emotional Bond Between Humans and Pets.txt'}),
 Document(page_content='Routine health care is cru

In [12]:
from langchain.embeddings.openai import OpenAIEmbeddings

In [13]:
from langchain.embeddings import SentenceTransformerEmbeddings

In [14]:

# embeddings = OpenAIEmbeddings(model_name="ada")

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [16]:
from langchain.vectorstores import Chroma

db=Chroma.from_documents(docs,embeddings)

In [17]:
query = "What are the different kinds of pets people commonly own?"
matching_docs = db.similarity_search(query)

In [18]:

matching_docs[0]

Document(page_content='Pet animals come in all shapes and sizes, each suited to different lifestyles and home environments. Dogs and cats are the most common, known for their companionship and unique personalities. Small mammals like hamsters, guinea pigs, and rabbits are often chosen for their low maintenance needs. Birds offer beauty and song, and reptiles like turtles and lizards can make intriguing pets. Even fish, with their calming presence, can be wonderful pets.', metadata={'source': '/content/drive/MyDrive/pets/Different Types of Pet Animals.txt'})

In [19]:

matching_docs[0].page_content

'Pet animals come in all shapes and sizes, each suited to different lifestyles and home environments. Dogs and cats are the most common, known for their companionship and unique personalities. Small mammals like hamsters, guinea pigs, and rabbits are often chosen for their low maintenance needs. Birds offer beauty and song, and reptiles like turtles and lizards can make intriguing pets. Even fish, with their calming presence, can be wonderful pets.'

In [20]:
matching_docs = db.similarity_search_with_score(query,k=2)
matching_docs

[(Document(page_content='Pet animals come in all shapes and sizes, each suited to different lifestyles and home environments. Dogs and cats are the most common, known for their companionship and unique personalities. Small mammals like hamsters, guinea pigs, and rabbits are often chosen for their low maintenance needs. Birds offer beauty and song, and reptiles like turtles and lizards can make intriguing pets. Even fish, with their calming presence, can be wonderful pets.', metadata={'source': '/content/drive/MyDrive/pets/Different Types of Pet Animals.txt'}),
  0.7325010299682617),
 (Document(page_content='Pets offer more than just companionship; they provide emotional support, reduce stress, and can even help their owners lead healthier lives. The bond between pets and their owners is strong, and many people consider their pets as part of the family. This bond can be especially important in times of personal or societal stress, providing comfort and consistency.', metadata={'source':

Persist a ChromaDB instance

In [22]:
persist_directory='chroma_db'
vectordb=Chroma.from_documents(documents=docs,embedding=embeddings,persist_directory=persist_directory)

In [23]:
vectordb.persist()

In [24]:

new_db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

In [25]:

matching_docs = new_db.similarity_search_with_score(query)
matching_docs[0]

(Document(page_content='Pet animals come in all shapes and sizes, each suited to different lifestyles and home environments. Dogs and cats are the most common, known for their companionship and unique personalities. Small mammals like hamsters, guinea pigs, and rabbits are often chosen for their low maintenance needs. Birds offer beauty and song, and reptiles like turtles and lizards can make intriguing pets. Even fish, with their calming presence, can be wonderful pets.', metadata={'source': '/content/drive/MyDrive/pets/Different Types of Pet Animals.txt'}),
 0.7325010124559873)

**LLM**

In [26]:
import os
os.environ["OPENAI_API_KEY"] ='your key'

In [27]:

from langchain.chat_models import ChatOpenAI
model_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=model_name)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


**Document QA**

In [28]:
from langchain.chains.question_answering import load_qa_chain
# chain = load_qa_chain(llm, chain_type="stuff")
chain = load_qa_chain(llm, chain_type="stuff",verbose=True)

In [29]:

query = "What are the emotional benefits of owning a pet?"
matching_docs = db.similarity_search(query)
answer =  chain.run(input_documents=matching_docs, question=query)
answer

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Pets offer more than just companionship; they provide emotional support, reduce stress, and can even help their owners lead healthier lives. The bond between pets and their owners is strong, and many people consider their pets as part of the family. This bond can be especially important in times of personal or societal stress, providing comfort and consistency.

Pet animals come in all shapes and sizes, each suited to different lifestyles and home environments. Dogs and cats are the most common, known for their companionship and unique personalities. Small mammals like hamsters, guinea pigs, and rabbits are often chosen for their low maintenance needs. Birds offer beauty and song, and reptiles like t

"Owning a pet can provide emotional support, reduce stress, and help improve mental health. Pets can offer companionship, comfort, and a sense of purpose. The bond between pets and their owners can bring joy, reduce feelings of loneliness, and even increase feelings of happiness and well-being. Additionally, the routine of caring for a pet can provide structure and consistency in one's life, which can be particularly beneficial during stressful times."

**Retrieval QA**

In [30]:

from langchain.chains import RetrievalQA
retrieval_chain = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=db.as_retriever())
retrieval_chain.run(query)

'Owning a pet can provide emotional support, reduce stress, and help their owners lead healthier lives. The bond between pets and their owners is strong, offering comfort and consistency, especially during times of personal or societal stress. Many people consider their pets as part of the family, which can bring a sense of companionship and joy. Additionally, caring for a pet can foster a sense of purpose and routine, leading to increased feelings of happiness and well-being.'